In [1]:
import os
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from modules.config import TMPDIR, DATADIR
from modules.utils import load_db, normalize_name, print_groupby

FileNotFoundError: [Errno 2] No such file or directory: 'archivenode_api_key.txt'

In [ ]:
db = load_db()
len(db.index)

In [ ]:
db.framework.value_counts()

In [ ]:
db.category.value_counts()

In [ ]:
# Load survey data
columns = ['sentiment_score','decisionmaking_agency','decisionmaking_trust','mission_accomplished','want_need','fulfillment']
questions = ['do you feel a sense of agency in the decision-making process?',
             'do you trust the decision-making process?',
             'does the DAO effectively accomplish its mission?',
             'do you feel wanted and/or needed by the DAO?',
             'does contributing to this DAO bring you a sense of fulfillment?']
df = pd.read_csv(os.path.join(TMPDIR, 'survey_responses.csv'))[['DAO', *columns]]
len(df.index)




In [ ]:
# Clean, group and display aggregate results for DAOs i 
df['daoName'] = df['DAO'].apply(normalize_name)
df['daoName'].unique()

In [ ]:
gb = df.groupby('daoName').agg({'daoName': ['count'],
                                  **{c: ['mean', 'median'] for c in columns}})
gb[gb['daoName']['count'] >= 5]

In [ ]:
# Compare with other data sources
df_gb = pd.DataFrame(gb)
df_gb.rename({'daoName': 'all'}, axis=1, inplace=True)
df_all = pd.merge(db, df_gb, on='daoName', how='right')
survey_columns = df_all.columns[7::2]

In [ ]:
gb_framework = df_all.groupby('framework').agg({'daoName': ['count'],
                                  **{c: ['mean', 'std'] for c in survey_columns}})
gb_framework = gb_framework[gb_framework['daoName']['count'] > 1]
gb_framework

In [ ]:
for c in gb_framework.columns[1::2]:
    df = gb_framework[c]
    plt.figure(figsize=(16, 6))
    sns.barplot(x=gb_framework.index, y=c, data=gb_framework, ci=None)
    plt.xlabel('Framework', fontsize=18)
    plt.savefig(os.path.join(TMPDIR, f"survey_vs_framework_{c}.svg"), dpi=300)
    plt.savefig(os.path.join(TMPDIR, f"survey_vs_framework_{c}.png"), dpi=300)

In [ ]:
gb_category = df_all.groupby('category').agg({'daoName': ['count'],
                                  **{c: ['mean'] for c in survey_columns}})
gb_category = gb_category[gb_category['daoName']['count'] > 1]
gb_category

In [ ]:
for c in gb_category.columns[1::2]:
    df = gb_category[c]
    plt.figure(figsize=(16, 6))
    sns.barplot(x=gb_category.index, y=c, data=gb_category, ci=None)
    plt.xlabel('Category', fontsize=18)
    plt.savefig(os.path.join(TMPDIR, "survey_vs_category.svg"), dpi=300)
    plt.savefig(os.path.join(TMPDIR, "survey_vs_category.png"), dpi=300)